In [2]:
import torch
from typing import Tuple
from torch import Tensor
from typing import Literal, Optional, Union

@torch.no_grad()
def broadcast(
    Q: Tensor,
    R: Tensor
) -> Tuple[Tensor, Tensor]:
    
    Q_expanded = Q.view(*Q.shape, *(1,)*R.ndim)
    R_expanded = R.view(*(1,)*Q.ndim, *R.shape)
    
    return Q_expanded, R_expanded

@torch.no_grad()
def get_delta_matrix(
    Q: Tensor,
    R: Tensor
) -> Tensor:
    
    return torch.abs(Q - R)

@torch.no_grad()
def ppm_convert(
    D: Tensor,
    R: Tensor,
) -> Tensor:
    return D * (1e6 / R)

@torch.no_grad()
def get_bool_matrix(
    D: Tensor,
    T: float
) -> Tensor:
    return D <= T

@torch.no_grad()
def get_indices(
    B: Tensor,
) -> Tensor:
    return B.nonzero(as_tuple=False)

@torch.no_grad()
def indices_offset(
    I: Tensor,
    qry_offset: int,
    ref_offset: int,
) -> Tensor:
    I[:, 0] += qry_offset
    I[:, 1] += ref_offset
    return I

@torch.no_grad()
def adduct_co_occurrence_filter(
    I: Tensor,
    threshold: int,
    dim: int,
    D: Tensor,
) -> Union[Tensor, Tuple[Tensor, Tensor]]:
    
    if I.size(0) == 0:
        return (I, D) if D is not None else I
    
    # 统计有效参考样本
    _, ref_counts = torch.unique(I[:, dim], return_counts=True)
    valid_mask = ref_counts[I[:, dim]] >= threshold
    
    # 同步过滤逻辑
    return I[valid_mask], D[valid_mask]

@torch.no_grad()
def mz_search_cpu(
    qry_ions: Tensor,
    ref_mzs: Tensor,
    mz_tolerance: float = 3,
    mz_tolerance_type: Literal['ppm','Da'] = 'ppm',
    query_RTs: Optional[Tensor] = None,
    ref_RTs: Optional[Tensor] = None,
    RT_tolerance: float = 0.1,
    adduct_co_occurrence_threshold: int = 1,
    chunk_size: int = 5120,
    work_device: torch.device = torch.device("cpu"),
    output_device: Optional[torch.device] = None,
) -> Tuple[Tensor, Tensor]:
    """改进的m/z搜索函数，支持设备管理和分块过滤"""
    output_device = output_device or work_device
    qry_ions = qry_ions.to(work_device)
    ref_mzs = ref_mzs.to(work_device)
    query_RTs = query_RTs.to(work_device) if query_RTs is not None else None
    ref_RTs = ref_RTs.to(work_device) if ref_RTs is not None else None

    all_indices, all_deltas = [], []
    q_dim = len(qry_ions.shape)

    for q_idx, q_chunk in enumerate(qry_ions.split(chunk_size)):
        q_offset = q_idx * chunk_size
        current_ref_offset = 0
        chunk_indices, chunk_deltas = [], []

        for r_chunk in ref_mzs.split(chunk_size):
            Q, R = broadcast(q_chunk.to(work_device), r_chunk.to(work_device))
            delta = torch.abs(Q - R)

            if mz_tolerance_type == 'ppm':
                delta = delta * (1e6 / R.clamp(min=1e-6))

            mz_mask = delta <= mz_tolerance

            if query_RTs is not None and ref_RTs is not None:
                rt_q = query_RTs[q_offset:q_offset+len(q_chunk)]
                rt_r = ref_RTs[current_ref_offset:current_ref_offset+len(r_chunk)]
                rt_q, rt_r = broadcast(rt_q, rt_r)
                rt_delta = torch.abs(rt_q - rt_r)
                mz_mask &= rt_delta <= RT_tolerance

            local_indices = mz_mask.nonzero(as_tuple=False)
            if local_indices.size(0) > 0:
                global_indices = indices_offset(local_indices, q_offset, current_ref_offset)
                chunk_indices.append(global_indices)
                chunk_deltas.append(delta[mz_mask])

            current_ref_offset += len(r_chunk)

        if chunk_indices:
            I_chunk = torch.cat(chunk_indices)
            D_chunk = torch.cat(chunk_deltas)

            if adduct_co_occurrence_threshold > 1:
                I_chunk, D_chunk = adduct_co_occurrence_filter(
                    I_chunk, adduct_co_occurrence_threshold, q_dim, D_chunk
                )

            all_indices.append(I_chunk)
            all_deltas.append(D_chunk)

    I = torch.cat(all_indices) if all_indices else torch.empty((0, 2), dtype=torch.long, device=work_device)
    D = torch.cat(all_deltas) if all_deltas else torch.empty((0,), dtype=ref_mzs.dtype, device=work_device)

    return I.to(output_device), D.to(output_device)

In [3]:
qry = torch.tensor([100.0, 200.0, 300.0])
ref = torch.tensor([100.003, 199.995, 305.0])
I, D = mz_search_cpu(qry, ref, mz_tolerance=50, mz_tolerance_type='ppm')

In [4]:
I

tensor([[0, 0],
        [1, 1]])

In [5]:
D

tensor([29.9826, 25.0250])

In [6]:
@torch.no_grad()
def mz_search_cuda(
    qry_ions: torch.Tensor,
    ref_mzs: torch.Tensor,
    mz_tolerance: float = 3,
    mz_tolerance_type: Literal['ppm','Da'] = 'ppm',
    query_RTs: Optional[torch.Tensor] = None,
    ref_RTs: Optional[torch.Tensor] = None,
    RT_tolerance: float = 0.1,
    adduct_co_occurrence_threshold: int = 1,
    chunk_size: int = 5120,
    work_device: torch.device = torch.device("cuda:0"),
    output_device: Optional[torch.device] = None,
) -> Tuple[torch.Tensor, torch.Tensor]:
    
    # 设备初始化
    output_device = output_device or work_device
    input_stream = torch.cuda.Stream()
    compute_stream = torch.cuda.Stream()
    output_stream = torch.cuda.Stream()
    
    # 数据预迁移
    with torch.cuda.stream(input_stream):
        qry_ions = qry_ions.to(work_device, non_blocking=True)
        ref_mzs = ref_mzs.to(work_device, non_blocking=True)
        query_RTs = query_RTs.to(work_device, non_blocking=True) if query_RTs is not None else None
        ref_RTs = ref_RTs.to(work_device, non_blocking=True) if ref_RTs is not None else None
    
    # 分块管理
    q_chunks = list(qry_ions.split(chunk_size))
    r_chunks = list(ref_mzs.split(chunk_size))
    
    # 流水线控制
    all_indices, all_deltas = [], []
    
    for q_idx, q_chunk in enumerate(q_chunks):
        q_offset = q_idx * chunk_size
        current_ref_offset = 0
        chunk_indices, chunk_deltas = [], []
        
        # 初始化参考分块流水线
        with torch.cuda.stream(input_stream):
            current_r = r_chunks[0].to(work_device, non_blocking=True)
        
        for r_idx in range(len(r_chunks)):
            # 预取下一分块
            next_r = r_chunks[r_idx+1] if r_idx+1 < len(r_chunks) else None
            if next_r is not None:
                with torch.cuda.stream(input_stream):
                    next_r = next_r.to(work_device, non_blocking=True)
            
            # 计算逻辑
            with torch.cuda.stream(compute_stream):
                torch.cuda.current_stream().wait_stream(input_stream)
                
                Q, R = broadcast(q_chunk, current_r)
                delta = torch.abs(Q - R)
                
                if mz_tolerance_type == 'ppm':
                    delta = delta * (1e6 / R.clamp(min=1e-6))
                
                mz_mask = delta <= mz_tolerance
                
                if query_RTs is not None and ref_RTs is not None:
                    rt_q = query_RTs[q_offset:q_offset+len(q_chunk)]
                    rt_r = ref_RTs[current_ref_offset:current_ref_offset+len(current_r)]
                    rt_q, rt_r = broadcast(rt_q, rt_r)
                    rt_delta = torch.abs(rt_q - rt_r)
                    mz_mask &= rt_delta <= RT_tolerance
                
                local_indices = mz_mask.nonzero(as_tuple=False)
                if local_indices.size(0) > 0:
                    global_indices = indices_offset(local_indices, q_offset, current_ref_offset)
                    chunk_indices.append(global_indices)
                    chunk_deltas.append(delta[mz_mask])
            
            # 结果回传
            with torch.cuda.stream(output_stream):
                torch.cuda.current_stream().wait_stream(compute_stream)
                if chunk_indices:
                    all_indices.append(torch.cat(chunk_indices).to(output_device, non_blocking=True))
                    all_deltas.append(torch.cat(chunk_deltas).to(output_device, non_blocking=True))
            
            current_ref_offset += current_r.size(0)
            current_r = next_r
        
        # 分块内加合物过滤
        with torch.cuda.stream(compute_stream):
            if adduct_co_occurrence_threshold > 1 and chunk_indices:
                I_chunk = torch.cat(chunk_indices)
                D_chunk = torch.cat(chunk_deltas)
                I_chunk, D_chunk = adduct_co_occurrence_filter(
                    I_chunk, adduct_co_occurrence_threshold, 
                    len(qry_ions.shape), D=D_chunk
                )
                chunk_indices = [I_chunk]
                chunk_deltas = [D_chunk]
    
    # 最终同步
    torch.cuda.synchronize()
    
    # 结果整合
    with torch.cuda.stream(output_stream):
        I = torch.cat(all_indices) if all_indices else torch.empty((0,2), dtype=torch.long, device=output_device)
        D = torch.cat(all_deltas) if all_deltas else torch.empty((0,), dtype=ref_mzs.dtype, device=output_device)
        
        # 维度校正
        if I.size(0) == 0:
            I = I.reshape(0, len(qry_ions.shape)+len(ref_mzs.shape))
            D = D.reshape(0)
    
    return I, D

In [7]:
qry = torch.tensor([100.0, 200.0, 300.0])
ref = torch.tensor([100.003, 199.995, 305.0])
I, D = mz_search_cuda(qry, ref, mz_tolerance=50, mz_tolerance_type='ppm')

In [8]:
I

tensor([[0, 0],
        [1, 1]], device='cuda:0')

In [9]:
D

tensor([29.9826, 25.0250], device='cuda:0')

In [ ]:
def resolve_device(
    device: Union[str, torch.device, Literal['auto']], 
    default: torch.device
) -> torch.device:
    if isinstance(device, torch.device):
        return device
    if device == 'auto':
        return default
    if device == 'cuda':
        device = 'cuda:0'
    return torch.device(device)

def mz_search(
    qry_ions: torch.Tensor,
    ref_mzs: torch.Tensor,
    mz_tolerance: float = 3,
    mz_tolerance_type: Literal['ppm','Da'] = 'ppm',
    query_RTs: Optional[torch.Tensor] = None,
    ref_RTs: Optional[torch.Tensor] = None,
    RT_tolerance: float = 0.1,
    adduct_co_occurrence_threshold: int = 1,
    chunk_size: int = 5120,
    work_device: Union[str, torch.device, Literal['auto']] = 'auto',
    output_device: Union[str, torch.device, Literal['auto']] = 'auto',
) -> Tuple[torch.Tensor, torch.Tensor]:
    
    # 设备自动推断
    _work_device = resolve_device(work_device, qry_ions.device)
    _output_device = resolve_device(output_device, _work_device)
    
    # 设备分发逻辑
    if _work_device.type.startswith('cuda'):
        return mz_search_cuda(
            qry_ions, ref_mzs, 
            mz_tolerance=mz_tolerance,
            mz_tolerance_type=mz_tolerance_type,
            query_RTs=query_RTs,
            ref_RTs=ref_RTs,
            RT_tolerance=RT_tolerance,
            adduct_co_occurrence_threshold=adduct_co_occurrence_threshold,
            chunk_size=chunk_size,
            work_device=_work_device,
            output_device=_output_device
        )
    else:
        return mz_search_cpu(
            qry_ions, ref_mzs,
            mz_tolerance=mz_tolerance,
            mz_tolerance_type=mz_tolerance_type,
            query_RTs=query_RTs,
            ref_RTs=ref_RTs,
            RT_tolerance=RT_tolerance,
            adduct_co_occurrence_threshold=adduct_co_occurrence_threshold,
            chunk_size=chunk_size,
            work_device=_work_device,
            output_device=_output_device
        )

In [11]:
qry = torch.tensor([100.0, 200.0, 300.0])
ref = torch.tensor([100.003, 199.995, 305.0])
I, D = mz_search(qry, ref, mz_tolerance=50, mz_tolerance_type='ppm')

In [12]:
I

tensor([[0, 0],
        [1, 1]])

In [13]:
D

tensor([29.9826, 25.0250])

In [14]:
qry = torch.tensor([100.0, 200.0, 300.0])
ref = torch.tensor([100.003, 199.995, 305.0])
I, D = mz_search(qry, ref, mz_tolerance=50, mz_tolerance_type='ppm',work_device='cuda:0')

In [15]:
I

tensor([[0, 0],
        [1, 1]], device='cuda:0')

In [16]:
D

tensor([29.9826, 25.0250], device='cuda:0')